In [2]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import sys
import glob
import matplotlib as plt
from random import shuffle

In [3]:
# load an image
def load(address):
    img = cv.imread(address)
    #make sure color is rgb
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return img

In [4]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def create_record(file_name,imgs,labels,class_ids):
    writer = tf.python_io.TFRecordWriter(file_name)
    for img,label,class_id in zip(imgs,labels,class_ids):
        if img is None:
            continue
        feature = {
            "image_raw": _bytes_feature(img.tostring()),
            'label': _int64_feature(label),
            'classes': _int64_feature([class_id])
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [20]:
path = 'data/*/*.png'
addrs = glob.glob(path)
label_path = 'data/*/*.txt'
label_files = [label for label in glob.glob(label_path) if 'base' in label]
class_ids = [1 for add in addrs]

In [37]:
print(addrs[768])

data\Testgame_22_12_17_Loki\26_01_2018__20_02_54_0697_upper.png


In [34]:
import csv

with open('data_annotated.csv', mode='w',newline='') as data_file:
    data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for add, label in zip(addrs,bounding_boxes):
        data_writer.writerow([add,label[0],label[1],label[2],label[3],"ball"])

In [33]:
with open('data_annotated.csv', mode='w') as data_file:
    data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in csv.reader(data_file):
        if row:
            data_write.writerow(row)

UnsupportedOperation: not readable

In [22]:
def create_coordinates(label_files):
    img_coordinates=[]
    for label_file in label_files:
        for line in open(label_file, 'r'):
            img_coordinates.append(line[41:-2])
    img_coordinates = [coor[:-2]+coor[-1] for coor in img_coordinates]
    img_coordinates = [coor[1:-1] for coor in img_coordinates]
    img_coordinates = [coor.split(",") for coor in img_coordinates]
    img_coordinates = [[{el[:4][1:-1]:int(el[5:][2:-1])} for el in coord] for coord in img_coordinates]
    for i in range(len(img_coordinates)):
        dall = {}
        for d in img_coordinates[i]:
            dall.update(d)
        img_coordinates[i] = dall
    return img_coordinates

In [23]:
bounding_boxes = create_coordinates(label_files)
bounding_boxes = [list(d.values()) for d in bounding_boxes]

In [12]:
print(len(meaned))

NameError: name 'meaned' is not defined

In [6]:
# mean subtraction en normalization of images over entire dataset
imgs = np.array([load(add) for add in addrs])
noised = [cv.fastNlMeansDenoisingColored(img, None, 15, 21, 7, 10) for img in imgs]
noised = np.array(noised).astype('uint8')
meaned = noised - np.mean(noised) 
#imgs /= np.std(imgs, axis = 0)


In [60]:
print([img.shape for img in imgs])

[(480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640, 3), (480, 640

In [27]:
c = list(zip(meaned, bounding_boxes,class_ids))
shuffle(c)
addrs, labels,ids = zip(*c)
    
# Divide the data into 60% train, 20% validation, and 20% test
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]
train_ids = ids[0:int(0.8*len(ids))]
val_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
val_ids = ids[int(0.8*len(ids)):int(0.9*len(ids))]
test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]
test_ids = ids[int(0.9*len(ids)):]

create_record('prep_train.tfrecords', train_addrs, train_labels, train_ids)
create_record('prep_val.tfrecords', val_addrs, val_labels, val_ids)
create_record('prep_test.tfrecords', test_addrs, test_labels, test_ids)

In [217]:
result=0
for example in tf.python_io.tf_record_iterator("train.tfrecords"):
    result += 1

In [218]:
print(result)

824


In [19]:
prepath = 'other_data/uitgesneden ballen/*.png'
preaddrs = glob.glob(prepath)
pre_imgs = [load(add) for add in preaddrs]
prelabels = [[0,0,img.shape[0],img.shape[1]] for img in pre_imgs]

In [90]:
print(prelabels)

[[0, 0, 23, 23], [0, 0, 23, 23], [0, 0, 27, 27], [0, 0, 27, 27], [0, 0, 19, 19], [0, 0, 19, 19], [0, 0, 24, 24], [0, 0, 24, 24], [0, 0, 39, 39], [0, 0, 39, 39], [0, 0, 19, 19], [0, 0, 19, 19], [0, 0, 36, 36], [0, 0, 36, 36], [0, 0, 36, 36], [0, 0, 36, 36], [0, 0, 35, 35], [0, 0, 35, 35], [0, 0, 38, 38], [0, 0, 38, 38], [0, 0, 33, 33], [0, 0, 33, 33], [0, 0, 35, 35], [0, 0, 35, 35], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 23, 23], [0, 0, 23, 23], [0, 0, 22, 22], [0, 0, 22, 22], [0, 0, 22, 22], [0, 0, 22, 22], [0, 0, 20, 20], [0, 0, 20, 20], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 22, 22], [0, 0, 22, 22], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 19, 19], [0, 0, 19, 19], [0, 0, 32, 32], [0, 0, 32, 32], [0, 0, 19, 19], [0, 0, 19, 19], [0, 0, 21, 21], [0, 0, 21, 21], [0, 0, 22, 22], [0, 0, 22, 22], [0, 0, 16, 16], [0, 0, 29, 29], [0, 0, 29, 29], [0, 0, 27, 27], [0, 0, 27, 27], [0, 0, 19, 19], [0, 0, 16, 16], [0, 0, 16, 16], [0, 0, 

In [91]:
c = list(zip(preaddrs, prelabels))
shuffle(c)
pre_addrs, pre_labels = zip(*c)
    
# Divide the data into 60% train, 20% validation, and 20% test
train_addrs = pre_addrs[0:int(0.6*len(pre_addrs))]
train_labels = pre_labels[0:int(0.6*len(pre_labels))]
val_addrs = pre_addrs[int(0.6*len(pre_addrs)):int(0.8*len(pre_addrs))]
val_labels = pre_labels[int(0.6*len(pre_labels)):int(0.8*len(pre_labels))]
test_addrs = pre_addrs[int(0.8*len(pre_addrs)):]
test_labels = pre_labels[int(0.8*len(pre_labels)):]

create_record('pre_train.tfrecords', train_addrs, train_labels)
create_record('pre_val.tfrecords', val_addrs, val_labels)
create_record('pre_test.tfrecords', test_addrs, test_labels)

-6.840812964636146e-13


In [240]:
meaned /= np.std(meaned,axis=0)

In [18]:
cv.imshow('dst_rt', imgs[144])
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('01.png',imgs[144])

True

In [36]:
count=0
for add in addrs:
    img = load(add)
    cv.imwrite(add,img)
    count+=1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [24]:
for example in tf.python_io.tf_record_iterator("train.tfrecords"):
    result = tf.train.Example.FromString(example)

In [25]:
print(result)

features {
  feature {
    key: "classes"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "image_raw"
    value {
      bytes_list {
        value: "VM}Y^\206NEjQC\207`G\246g8\201eItNB\215IEnHQ\213IK\216ZF\206PM{?H\204QL\207aJ\217lJ\215WJv:E\2179/o$$$\035\035\035\027\027\027\023\023\023\022\022\022\022\022\022\023\023\023\022\022\022\022\022\022\022\022\022\023\023\023\025\025\025\022 \033\0246b\'E|/P\2134T\230AV\206IX\2079+U&&)\027 \036\030\030\030\024\024\024\022\022\022\021\021\021\021\021\021\022\022\022\022\022\022\021\021\021\021\021\021\021\021\021\021\021\021\022\022\022\022\022\022\022\022\022\025\025\025\022\022\022\021\021\021\021\021\021\021\021\021\022\022\022\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\022\022\022\021\021\021\021\021\021\021\021\021\022\022\022\021\021\021\021\021\021\022\022\022\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021\021